In [41]:
from PIL import Image
import os
# https://pillow.readthedocs.io/en/stable/handbook/tutorial.html

In [42]:
ROOT_OF_IMAGES = './ppcovers/'
REGION_FOLDERS = ["africa","oceania","asia","south_america","north_america","europe"]


region_to_full_pp_cover_paths = {}
for region_name in REGION_FOLDERS:
  region_folder_path = os.path.join(ROOT_OF_IMAGES,region_name)
  pp_images_in_folder = os.listdir(region_folder_path)
  full_image_paths_for_region = [os.path.join(region_folder_path,pp_image_filename) for pp_image_filename in pp_images_in_folder]
  region_to_full_pp_cover_paths[region_name] = full_image_paths_for_region
region_to_full_pp_cover_paths


{'africa': ['./ppcovers/africa/Kenya.png',
  './ppcovers/africa/Ghana.png',
  './ppcovers/africa/Sao+Tome+and+Principe.png',
  './ppcovers/africa/Namibia.png',
  './ppcovers/africa/Tanzania.png',
  './ppcovers/africa/Djibouti.png',
  './ppcovers/africa/Angola.png',
  './ppcovers/africa/Tunisia.png',
  './ppcovers/africa/Algeria.png',
  './ppcovers/africa/Seychelles.png',
  './ppcovers/africa/Mozambique.png',
  './ppcovers/africa/Benin.png',
  './ppcovers/africa/Gabon.png',
  './ppcovers/africa/Libya.png',
  './ppcovers/africa/Cape+Verde.png',
  './ppcovers/africa/Madagascar.png',
  './ppcovers/africa/Senegal.png',
  './ppcovers/africa/Comoros.png',
  './ppcovers/africa/South+Africa.png',
  './ppcovers/africa/Togo.png',
  './ppcovers/africa/Mauritania.png',
  './ppcovers/africa/Somalia.png',
  './ppcovers/africa/Uganda.png',
  './ppcovers/africa/Guinea-Bissau.png',
  './ppcovers/africa/Sudan.png',
  './ppcovers/africa/Eswatini.png',
  './ppcovers/africa/Chad.png',
  './ppcovers/africa/E

In [43]:
from collections import defaultdict

image_sizes = defaultdict(lambda :  0)
image_aspect_ratios = defaultdict(lambda :  0)
for region in region_to_full_pp_cover_paths:
  for image_path in region_to_full_pp_cover_paths[region]:
    image = Image.open(image_path)
    image_sizes[image.size] += 1
    image_aspect_ratios[image.size] += 1
    if image.size == (1087,1542):
      image.show()


for image_size in image_sizes:
  print(f"{image_size}:{image_sizes[image_size]}")
# print(image_aspect_ratios)
print(f"min image size: {min([size[0] for size in image_sizes.keys()])} ")


(667, 1000):23
(674, 1000):41
(705, 1000):52
(707, 1000):17
(719, 1000):7
(718, 1000):1
(710, 1000):1
(696, 1000):1
(1087, 1542):1
min image size: 667 


The region is defined by a 4-tuple, where coordinates are (left, upper, right, lower). The Python Imaging Library uses a coordinate system with (0, 0) in the upper left corner. Also note that coordinates refer to positions between the pixels, so the region in the above example is exactly 300x300 pixels.

In [44]:
# box should be (667 wide, and 10000 tall)

def get_box_for_image_path(image_path):
  desired_box_width = 667
  desired_box_height = 1000   
  image = Image.open(image_path)
  (image_width, image_height) = image.size
  extra_width_pixels = image_width - desired_box_width
  extra_height_pixels = image_height - desired_box_height
  from_left_box_start = int(extra_width_pixels / 2)
  right_end = from_left_box_start + desired_box_width
  from_top_box_start = int(extra_height_pixels / 2)
  bottom_end = from_top_box_start + desired_box_height  
  
  box = (from_left_box_start, from_top_box_start, right_end, bottom_end)
  print(f"Box is {box}")
  return box



# for region in region_to_full_pp_cover_paths:
#   for image_path in region_to_full_pp_cover_paths[region]:

swiss_reisenpass = Image.open("./ppcovers/europe/Switzerland.png")
swiss_reisenpass.show()
box = get_box_for_image_path("./ppcovers/europe/Switzerland.png")
resize = swiss_reisenpass.resize((705, 1000))
print(f"Region shape is {resize.size}")
resize.show()

Box is (210, 271, 877, 1271)
Region shape is (705, 1000)


In [45]:
def merge_horizontally(im1, im2):
    w = im1.size[0] + im2.size[0]
    h = max(im1.size[1], im2.size[1])
    im = Image.new("RGBA", (w, h))

    im.paste(im1)
    im.paste(im2, (im1.size[0], 0))

    return im

def merge_vertically(im1, im2):
    h = im1.size[1] + im2.size[1]
    w = max(im1.size[0], im2.size[0])
    im = Image.new("RGBA", (w, h))

    im.paste(im1)
    im.paste(im2, (0, im1.size[1]))

    return im

In [48]:
def get_poster(images_per_row):
  elements_in_row_so_far = 0
  poster = None
  all_pp_paths = []
  for region_paths in region_to_full_pp_cover_paths.values():
    for path in region_paths:
      all_pp_paths.append(path)
  for image_path in all_pp_paths:
    pp_image = Image.open(image_path)
    resized_image = pp_image.resize((705, 1000))
    if poster is None:
      poster = resized_image
    else:
      if elements_in_row_so_far < images_per_row:
        poster = merge_horizontally(poster,resized_image)
        elements_in_row_so_far += 1
      else:
        # start new row
        break
        elements_in_row_so_far = 0
        poster = merge_vertically(poster,resized_image)

  return poster

poster = get_poster(10)
poster.show()